In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaForSequenceClassification, RobertaTokenizer
import pandas as pd
import numpy as np

emotion_to_label = {
    0: 'anger', 1: 'disgust', 2: 'fear', 3: 'sadness', 4: 'contentment', 5: 'excitement', 6: 'awe', 7: 'amusement'
}

In [ ]:
# CUDA 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
# RoBERTa 모델 및 토크나이저 불러오기
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=len(emotion_to_label))
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# 모델을 GPU로 이동
model.to(device)
model.eval()

# 불러올 모델 파일 경로
model_path = "/content/drive/MyDrive/앤트 공유 문서함/0-1.감정인식/Model/0ver2_SOTA_ED_model_comp_0.8973.pt"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
# 저장된 모델의 state_dict 불러오기
state_dict = torch.load(model_path)

# 모델에 state_dict 적용
model.load_state_dict(state_dict)

<All keys matched successfully>

In [ ]:
# CSV 파일을 pandas DataFrame으로 읽기
test_data = pd.read_csv("/content/drive/MyDrive/앤트 공유 문서함/0-2-2.main/dataset/T5_T2_v2.csv")

In [ ]:
# 데이터셋 클래스 정의
class EmotionDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        self.data = pd.DataFrame(data)
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        utterance = self.data['original_text'].iloc[idx]
        text = f"{utterance}"

        encoding = self.tokenizer(text, max_length=self.max_length, padding='max_length', truncation=True, return_tensors='pt')

        return {
            'input_ids': encoding['input_ids'],
            'attention_mask': encoding['attention_mask']
        }

test_dataset = EmotionDataset(test_data, tokenizer)

In [ ]:
# 3. 예측 수행
def predict_label(inputs):
    with torch.no_grad():
        inputs['input_ids'] = inputs['input_ids'].to(device)
        inputs['attention_mask'] = inputs['attention_mask'].to(device)
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_label_id = logits.argmax(dim=1).item()
    predicted_label = emotion_to_label[predicted_label_id]
    return predicted_label

In [ ]:
predictions = [predict_label(inputs) for inputs in test_dataset]

In [ ]:
print(predictions)

['disgust', 'disgust', 'anger', 'anger', 'fear', 'disgust', 'anger', 'disgust', 'anger', 'disgust', 'anger', 'anger', 'anger', 'anger', 'anger', 'anger', 'disgust', 'anger', 'anger', 'disgust', 'anger', 'anger', 'anger', 'anger', 'disgust', 'anger', 'disgust', 'anger', 'anger', 'anger', 'disgust', 'disgust', 'fear', 'anger', 'disgust', 'anger', 'anger', 'sadness', 'disgust', 'anger', 'disgust', 'disgust', 'anger', 'disgust', 'anger', 'disgust', 'anger', 'fear', 'anger', 'anger', 'anger', 'sadness', 'anger', 'disgust', 'anger', 'disgust', 'anger', 'fear', 'anger', 'anger', 'disgust', 'anger', 'anger', 'anger', 'disgust', 'anger', 'anger', 'anger', 'anger', 'fear', 'anger', 'disgust', 'anger', 'disgust', 'anger', 'disgust', 'anger', 'anger', 'disgust', 'anger', 'anger', 'disgust', 'fear', 'anger', 'anger', 'anger', 'anger', 'disgust', 'disgust', 'anger', 'disgust', 'anger', 'anger', 'anger', 'anger', 'anger', 'anger', 'anger', 'anger', 'anger', 'anger', 'anger', 'disgust', 'anger', 'ange

In [ ]:
test_data['emotion'] = predictions

문장과 유사도 계산해 라벨 3개 추가

In [ ]:
import json

# JSON 파일에서 데이터 불러오기
with open('/content/drive/MyDrive/앤트 공유 문서함/1-1.감정라벨append/emotion_synonyms.json', 'r', encoding='utf-8') as file:
    synonyms = json.load(file)

In [ ]:
!pip install angle_emb faiss_cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.3/195.3 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 77.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━

In [ ]:
from angle_emb import AnglE

uae = AnglE.from_pretrained('WhereIsAI/UAE-Large-V1', pooling_strategy='cls').cuda()

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

In [ ]:
import faiss
from boltons.iterutils import chunked
from tqdm import tqdm

In [ ]:
def get_index(emotion):
    np.random.seed(1234)
    d = 1024  # 차원
    index = faiss.IndexFlatL2(d)  # 인덱스 생성

    chunk_size = 64
    synonyms_list = synonyms.get(emotion, [])

    for chunk in tqdm(chunked(synonyms_list, chunk_size)):
        for vec in uae.encode(chunk):
            index.add(vec.reshape(-1, d))

    return index

In [ ]:
def get_top3_synonyms(emotion, text):
    vec = uae.encode(text)
    synonyms_list = synonyms.get(emotion, [])
    append_synonyms = ''

    D, I = get_index(emotion).search(vec, 3)
    for id in I[0]:
      if append_synonyms=='':
        append_synonyms += synonyms_list[id]
      else: append_synonyms += ', ' + synonyms_list[id]

    print(append_synonyms)

    return append_synonyms

In [ ]:
test_data['prompt'] = test_data.apply(lambda row: f"{row['utterance']}, {row['emotion']}, {get_top3_synonyms(row['emotion'], row['original_text'])}", axis=1)
# test_data['prompt'] = test_data.apply(lambda row: f"{row['emotion']},{get_top3_synonyms(row['emotion'], row['original_text'])}, {row['utterance']}", axis=1)

100%|██████████| 1/1 [00:00<00:00,  5.90it/s]


blame, nauseate, awfulness


100%|██████████| 1/1 [00:00<00:00, 27.47it/s]


gross out, awfulness, sicken


100%|██████████| 1/1 [00:00<00:00, 16.49it/s]


...

afraidness, fearfulness, angst


100%|██████████| 1/1 [00:00<00:00, 30.56it/s]


afraidness, concern, fearfulness


100%|██████████| 1/1 [00:00<00:00, 20.84it/s]

creepiness, fright, afraidness


In [ ]:
test_data.to_csv('/content/drive/MyDrive/E4_감정동의어_다뒤로.csv', index=False)